In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
race_x = pd.read_pickle("data/race_x.pkl").values
race_y = pd.read_pickle("data/race_y.pkl").values
len(race_y)

57768

In [3]:
print(len(race_x[0]))
print(race_y[0])

29
1.0


In [4]:
# split data into train and test
x_train, x_test, y_train, y_test = train_test_split(race_x, race_y, train_size=0.7)

In [5]:
def learningModel(dim_1, dim_2, dim_3, batch_size, epochs):
    n_in = len(race_x[0])  # 入力層のニューロン数：　特徴量の数
    n_out = 1 # 出力層のニューロン数： 二値分類なので１

    # 入力層、中間層、出力層の３層のニューラルネットワークを構築する
    model = Sequential()

    #中間層を追加する。最初に層を追加するときはinput_shapeを設定すること。
    #input_shapeのところには形状を入れるため
    # 活性化関数にrelu関数
    model.add(Dense(dim_1, input_dim=n_in, activation="relu"))
    model.add(Dense(dim_2, input_dim=dim_1, activation="relu"))
    model.add(Dense(dim_3, input_dim=dim_2, activation="relu"))

    #出力層を追加する
    model.add(Dense(n_out, activation="sigmoid"))  

    # 損失関数にクロスエントロピー誤差、最適化アルゴリズムにRMSpropを使用する
    model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])  

    #学習開始
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    score, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
    print("Test score", score)
    print("Test accuracy", accuracy)
    return [model, history, score, accuracy]

In [6]:
model, history, score, accuracy = learningModel(20, 17, 19, 128, 20)

Epoch 1/20
316/316 [==============================] - 1s 491us/step - loss: 0.7382 - accuracy: 0.6076
Epoch 2/20
316/316 [==============================] - 0s 474us/step - loss: 0.6330 - accuracy: 0.6448
Epoch 3/20
316/316 [==============================] - 0s 470us/step - loss: 0.6190 - accuracy: 0.6628
Epoch 4/20
316/316 [==============================] - 0s 470us/step - loss: 0.6112 - accuracy: 0.6719
Epoch 5/20
316/316 [==============================] - 0s 472us/step - loss: 0.6051 - accuracy: 0.6801
Epoch 6/20
316/316 [==============================] - 0s 471us/step - loss: 0.6024 - accuracy: 0.6833
Epoch 7/20
316/316 [==============================] - 0s 477us/step - loss: 0.6004 - accuracy: 0.6843
Epoch 8/20
316/316 [==============================] - 0s 480us/step - loss: 0.5986 - accuracy: 0.6860
Epoch 9/20
316/316 [==============================] - 0s 473us/step - loss: 0.5965 - accuracy: 0.6888
Epoch 10/20
316/316 [==============================] - 0s 477us/step - loss: 0.596

In [7]:
def plot_history(history):
    #精度の履歴をプロット
    plt.plot(history.history["accuracy"], "o-", label="acc")
    plt.title("model_accuracy")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.legend(loc="lower right")
    plt.show()
    
    #損失の履歴をプロット
    plt.plot(history.history["loss"], "o-", label="loss")
    plt.title("model_loss")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.legend(loc="lower right")
    plt.show()

In [8]:
plot_history(history)

In [9]:
y_pred = model.predict(x_test)
print(y_pred[0])

[0.7516227]


In [10]:
#予測値を二値に変換する関数
def toBinary(x):
    if x >= 0.50:
        x = 1
    else:
        x = 0
    return x

In [11]:
y_pred = np.apply_along_axis(lambda x: toBinary(x), 1, y_pred)

In [13]:
#混同行列を見てみる
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(confusion_matrix)
print("Precision:", precision)
print("Recall", recall)
print("F1Score", f1)
confusion_matrix(y_test, y_pred)

<function confusion_matrix at 0x0000028162F4EDC0>
Precision: 0.6750793192962216
Recall 0.6167325428194993
F1Score 0.6445882676948499


array([[7488, 2253],
       [2909, 4681]], dtype=int64)

In [14]:
#学習モデルを保存
model.save("pred_top3")

INFO:tensorflow:Assets written to: pred_top3\assets
